<a href="https://colab.research.google.com/github/kanru-wang/coursera_quantization_pruning_distillation/blob/main/TFX_Tuner_and_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter tuning and model training with TFX

Doing hyperparameter tuning within a Tensorflow Extended (TFX) pipeline. 

<img src='https://www.tensorflow.org/tfx/guide/images/prog_trainer.png' alt='tfx pipeline'>

https://www.tensorflow.org/tfx/guide

The *Tuner* utilizes the Keras Tuner API under the hood to tune a model's hyperparameters.



## Setup

### Install TFX

In [ ]:
!pip install -U pip
!pip install tfx==1.12.0

**In Google Colab, need to restart the runtime at this point to finalize updating the packages just installed. Click the `Restart Runtime` at the end of the output cell above (after installation), or by selecting `Runtime > Restart Runtime` in the Menu bar.**

### Imports

In [ ]:
import os
import pprint

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from absl import logging

from tfx import v1 as tfx
from tfx.proto import example_gen_pb2, trainer_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propagate = False
tf.get_logger().setLevel('ERROR')
pp = pprint.PrettyPrinter()
logging.set_verbosity(logging.ERROR)

## Download and prepare the dataset

In [ ]:
# Location of the pipeline metadata store
_pipeline_root = './pipeline/'

# Directory of the raw data files
_data_root = './data/fmnist'

# Temporary directory
tempdir = './tempdir'

In [ ]:
# Create the dataset directory
!mkdir -p {_data_root}

# Create the TFX pipeline files directory
!mkdir {_pipeline_root}

In [ ]:
# Download the dataset
ds, ds_info = tfds.load('fashion_mnist', data_dir=tempdir, with_info=True)

In [ ]:
# Display info about the dataset
print(ds_info)

In [ ]:
# Define the location of the train tfrecord downloaded via TFDS
tfds_data_path = f'{tempdir}/{ds_info.name}/{ds_info.version}'

# Display contents of the TFDS data directory
os.listdir(tfds_data_path)

Copy the train split so it can be consumed by the ExampleGen component

In [ ]:
# Define the train tfrecord filename
train_filename = 'fashion_mnist-train.tfrecord-00000-of-00001'

# Copy the train tfrecord into the data root folder
!cp {tfds_data_path}/{train_filename} {_data_root}

## TFX Pipeline 

### Initialize the Interactive Context

In [ ]:
# Initialize the InteractiveContext
context = InteractiveContext(pipeline_root=_pipeline_root)

### ExampleGen

ExampleGen is the initial input component of a pipeline that ingests and optionally splits the input dataset. ImportExampleGen consumes TFRecords.

In [ ]:
# Specify 80/20 split for the train and eval set
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
    ])
)

# Ingest the data through ExampleGen
example_gen = tfx.components.ImportExampleGen(input_base=_data_root, output_config=output)

# Run the component
context.run(example_gen)

In [ ]:
# Print split names and URI
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

### StatisticsGen

StatisticsGen calculates statistics for the dataset.

In [ ]:
# Run StatisticsGen
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples']
)

context.run(statistics_gen)

### SchemaGen

SchemaGen infers a data schema, and validates incoming data to ensure that it is formatted correctly.

In [ ]:
# Run SchemaGen
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)

In [ ]:
# Visualize the results
context.show(schema_gen.outputs['schema'])

### ExampleValidator

ExampleValidator looks for anomalies and missing values in the dataset.

In [ ]:
# Run ExampleValidator
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

In [ ]:
# Visualize the results. There should be no anomalies.
context.show(example_validator.outputs['anomalies'])

### Transform

Transform performs feature engineering on the dataset.

In [ ]:
_transform_module_file = 'fmnist_transform.py'

In [ ]:
%%writefile {_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Keys
_LABEL_KEY = 'label'
_IMAGE_KEY = 'image'


def _transformed_name(key):
    return key + '_xf'

def _image_parser(image_str):
    '''converts the images to a float tensor'''
    image = tf.image.decode_image(image_str, channels=1)
    image = tf.reshape(image, (28, 28, 1))
    image = tf.cast(image, tf.float32)
    return image


def _label_parser(label_id):
    '''converts the labels to a float tensor'''
    label = tf.cast(label_id, tf.float32)
    return label


def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    
    # Convert the raw image and labels to a float array
    with tf.device("/cpu:0"):
        outputs = {
            _transformed_name(_IMAGE_KEY):
                tf.map_fn(
                    _image_parser,
                    tf.squeeze(inputs[_IMAGE_KEY], axis=1),
                    dtype=tf.float32),
            _transformed_name(_LABEL_KEY):
                tf.map_fn(
                    _label_parser,
                    inputs[_LABEL_KEY],
                    dtype=tf.float32)
        }
    
    # scale the pixels from 0 to 1
    outputs[_transformed_name(_IMAGE_KEY)] = tft.scale_to_0_1(outputs[_transformed_name(_IMAGE_KEY)])
    
    return outputs

Pass in the examples, schema, and transform module file.

Ignore the warnings and `udf_utils` related errors.

In [ ]:
# Setup the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_transform_module_file)
)

# Run the component
context.run(transform)

### Tuner

Prepare a *tuner module file* which contains a `tuner_fn()` function. 

In `_input_fn()`, the transformed examples as TFRecords compressed in `.gz` format are loaded into the memory. Once loaded, create batches of features and labels for hypertuning.

`tuner_fn()` returns a `TunerFnResult` tuple containing the `tuner` object and a set of arguments to pass to `tuner.search()` method.

In [ ]:
# Declare name of module file
_tuner_module_file = 'tuner.py'

In [ ]:
%%writefile {_tuner_module_file}

# Define imports
from kerastuner.engine import base_tuner
import kerastuner as kt
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# Declare namedtuple field names
TunerFnResult = NamedTuple(
    'TunerFnResult',
    [('tuner', base_tuner.BaseTuner), ('fit_kwargs', Dict[Text, Any])]
)

# Input key
_IMAGE_KEY = 'image_xf'

# Label key
_LABEL_KEY = 'label_xf'

# Callback for the search strategy
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


def _gzip_reader_fn(filenames):
    """Load compressed dataset
  
    Args:
        filenames - filenames of TFRecords to load

    Returns:
        TFRecordDataset loaded from the filenames
    """

    # Load the dataset. Specify the compression type since it is saved as `.gz`
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
    """Create batches of features and labels from TF Records

    Args:
        file_pattern - List of files or patterns of file paths containing Example records.
        tf_transform_output - transform output graph
        num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
        batch_size - An int representing the number of records to combine in a single batch.

    Returns:
        A dataset of dict elements, (or a tuple of dict elements and label). 
        Each dict maps feature keys to Tensor or SparseTensor objects.
    """

    # Get feature specification based on transform output
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()
  
    # Create batches of features and labels
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=_LABEL_KEY
    )
  
    return dataset


def model_builder(hp):
    """
    Builds the model and sets up the hyperparameters to tune.

    Args:
        hp - Keras tuner object

    Returns:
        model with hyperparameters to tune
    """

    # Initialize the Sequential API and start stacking the layers
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(28, 28, 1), name=_IMAGE_KEY))
    model.add(keras.layers.Flatten())

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu', name='dense_1'))

    # Add next layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """Build the tuner using the KerasTuner API.
    Args:
        fn_args: Holds args as name/value pairs.

        - working_dir: working dir for tuning.
        - train_files: List of file paths containing training tf.Example data.
        - eval_files: List of file paths containing eval tf.Example data.
        - train_steps: number of train steps.
        - eval_steps: number of eval steps.
        - schema_path: optional schema of the input data.
        - transform_graph_path: optional transform graph produced by TFT.
    
    Returns:
        A namedtuple contains the following:
            - tuner: A BaseTuner that will be used for tuning.
            - fit_kwargs: Args to pass to tuner's run_trial function for fitting
                the model , e.g., the training and validation dataset. Required
                args depend on the above tuner's implementation.
    """

    # Define tuner search strategy
    tuner = kt.Hyperband(
        model_builder,
        objective='val_accuracy',
        max_epochs=10,
        factor=3,
        directory=fn_args.working_dir,
        project_name='kt_hyperband'
    )

    # Load transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Use _input_fn() to extract input features and labels from the train and val set
    train_set = _input_fn(fn_args.train_files[0], tf_transform_output)
    val_set = _input_fn(fn_args.eval_files[0], tf_transform_output)

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={ 
            "callbacks":[stop_early],
            'x': train_set,
            'validation_data': val_set,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps
        }
    )

Pass a `num_steps` argument to the train and eval args and it is used in the `steps_per_epoch` and `validation_steps` arguments in the tuner module above. This can be useful for avoiding going through the entire dataset when tuning. For example, training data is very large, it would be incredibly time consuming to iterate through it entirely just for one epoch and one set of hyperparameters. Set the number of steps so to only go through a fraction of the dataset.

Total number of steps in one epoch = `number of examples / batch size`. In this example, `48000 examples / 32 (default size)` = `1500` steps per epoch for the train set (compute val steps from 12000 examples). Since `500` is passed to the `num_steps` of the train args, this means that some examples will be skipped. This will likely result in lower accuracy readings but will save time in doing the hypertuning.

In [ ]:
# Setup the Tuner component
tuner = tfx.components.Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
)

In [ ]:
# Run the component. This will take around 10 minutes to run.
# When done, it will summarize the results and show the 10 best trials.
context.run(tuner, enable_cache=False)

### Trainer

The Trainer component looks for a `run_fn()` function that defines and trains the model. 

Get the best result of the Tuner component through `fn_args.hyperparameters`, and pass it into `model_builder()`. Alternatively, just explicitly define the number of hidden units and learning rate.

In [ ]:
# Declare trainer module file
_trainer_module_file = 'trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# Input key
_IMAGE_KEY = 'image_xf'

# Label key
_LABEL_KEY = 'label_xf'

def _gzip_reader_fn(filenames):
    """Load compressed dataset
    
    Args:
        filenames - filenames of TFRecords to load

    Returns:
        TFRecordDataset loaded from the filenames
    """

    # Load the dataset. Specify the compression type since it is saved as `.gz`
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
    """Create batches of features and labels from TF Records

    Args:
        file_pattern - List of files or patterns of file paths containing Example records.
        tf_transform_output - transform output graph
        num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
        batch_size - An int representing the number of records to combine in a single batch.

    Returns:
        A dataset of dict elements, (or a tuple of dict elements and label). 
        Each dict maps feature keys to Tensor or SparseTensor objects.
    """
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=_LABEL_KEY
    )

    return dataset


def model_builder(hp):
    """
    Builds the model and sets up the hyperparameters to tune.

    Args:
        hp - Keras tuner object

    Returns:
        model with hyperparameters to tune
    """

    # Initialize the Sequential API and start stacking the layers
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(28, 28, 1), name=_IMAGE_KEY))
    model.add(keras.layers.Flatten())

    # Get the number of units from the Tuner results
    hp_units = hp.get('units')
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))

    # Add next layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10, activation='softmax'))

    # Get the learning rate from the Tuner results
    hp_learning_rate = hp.get('learning_rate')

    # Setup model for training
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    # Print the model summary
    model.summary()
    
    return model


def run_fn(fn_args: FnArgs) -> None:
    """Defines and trains the model.
    Args:
        fn_args: Holds args as name/value pairs. Refer here for the complete attributes: 
        https://www.tensorflow.org/tfx/api_docs/python/tfx/components/trainer/fn_args_utils/FnArgs#attributes
    """

    # Callback for TensorBoard
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir,
        update_freq='batch'
    )
    
    # Load transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data good for 10 epochs
    train_set = _input_fn(fn_args.train_files[0], tf_transform_output, 10)
    val_set = _input_fn(fn_args.eval_files[0], tf_transform_output, 10)

    # Load best hyperparameters
    hp = fn_args.hyperparameters.get('values')

    # Build the model
    model = model_builder(hp)

    # Train the model
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback]
    )
    
    # Save the model
    model.save(fn_args.serving_model_dir, save_format='tf')

In [ ]:
# Setup the Trainer component
trainer = tfx.components.Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

When re-training your model, don't always have to re-tune hyperparameters. Can import it with the ImporterNode.

```
hparams_importer = ImporterNode(
    instance_name='import_hparams',
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri='path/to/best_hyperparameters.txt',
    artifact_type=HyperParameters
)

trainer = Trainer(
    ...
    # An alternative is directly use the tuned hyperparameters in Trainer's user
    # module code and set hyperparameters to None here.
    hyperparameters=hparams_importer.outputs['result']
)
```

In [ ]:
# Run the component
context.run(trainer, enable_cache=False)

The file is saved as `saved_model.pb`

In [ ]:
# Get artifact uri of trainer model output
model_artifact_dir = trainer.outputs['model'].get()[0].uri

# List subdirectories artifact uri
print(f'contents of model artifact directory:{os.listdir(model_artifact_dir)}')

# Define the model directory
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')

# List contents of model directory
print(f'contents of model directory: {os.listdir(model_dir)}')

Visualize the training results by loading the logs saved by the Tensorboard callback.

In [ ]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}